In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Load dataset
data = pd.read_csv("winners_f1_1950_2025_v2.csv")

# Convert race time (HH:MM:SS) to seconds
def time_to_seconds(t):
    try:
        parts = list(map(int, t.split(":")))
        return parts[0] * 3600 + parts[1] * 60 + parts[2]
    except:
        return None

data["time_seconds"] = data["time"].apply(time_to_seconds)
data = data.dropna(subset=["time_seconds"])  # remove rows without valid times

# Features (multiple predictors)
X = data[["laps", "year", "grand_prix", "circuit", "team"]]
y = data["time_seconds"]

# Preprocess categorical variables
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), ["grand_prix", "circuit", "team"])
    ], remainder='passthrough')

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Multiple Linear Regression
lrr = LinearRegression()
X_train_enc = preprocessor.fit_transform(X_train)
X_test_enc = preprocessor.transform(X_test)

lrr.fit(X_train_enc, y_train)
y_pred = lrr.predict(X_test_enc)

# Evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Linear Regression MSE:", mse)
print("Linear Regression R2:", r2)


Linear Regression MSE: 837874.4107846403
Linear Regression R2: 0.711403112118626
